In [6]:
#ANÁLISE FATORIAL - ÍNDICES

# 1. Instalação e Importações
!pip install factor_analyzer pandas numpy scipy -q

import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity
from scipy.stats import multivariate_normal
from IPython.display import display, HTML
from google.colab import files

# 2. CONFIGURAÇÃO DA SIMULAÇÃO (N=500, Seed=42)
np.random.seed(42)
N = 500
total_q = 41

# Mapeamento Teórico (Itens determinantes para AFE)
items_isp = ['Q1', 'Q2', 'Q3', 'Q4', 'Q6', 'Q30', 'Q31']
items_iat = ['Q15', 'Q16', 'Q17', 'Q18', 'Q20', 'Q21', 'Q22']
items_nsp = ['Q7', 'Q8', 'Q9', 'Q24', 'Q25', 'Q27']
determinants = items_isp + items_iat + items_nsp

all_q_names = [f'Q{i}' for i in range(1, total_q + 1)]

# Matriz de Correlação Teórica para os 41 itens
cor_matrix = np.eye(total_q)
for items in [items_isp, items_iat, items_nsp]:
    idx = [all_q_names.index(q) for q in items]
    for i in idx:
        for j in idx:
            if i != j: cor_matrix[i, j] = 0.65

# 3. GERAÇÃO DOS DADOS
data_sim = multivariate_normal.rvs(mean=[3]*total_q, cov=cor_matrix, size=N)
df = pd.DataFrame(data_sim, columns=all_q_names).apply(lambda x: np.clip(np.round(x), 1, 5)).astype(int)

# 4. ANÁLISE FATORIAL EXPLORATÓRIA (PAF + PROMAX)
fa = FactorAnalyzer(n_factors=3, rotation='promax', method='principal')
fa.fit(df[determinants])

# --- 5. FUNÇÕES DE ESTILIZAÇÃO PARA O COLAB ---
def stylize(df, title):
    return df.style.set_caption(f"<div style='font-size:16px; font-weight:bold; color:black; padding:10px;'>{title}</div>")\
        .format(precision=3)\
        .map(lambda v: 'font-weight: bold; color: #2E86C1;' if abs(v) >= 0.3 else 'color: #999;')\
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f4f4f4'), ('border', '1px solid #ddd')]},
            {'selector': 'td', 'props': [('border', '1px solid #ddd'), ('text-align', 'center')]}
        ])

# 6. EXIBIÇÃO DOS RESULTADOS

# Tabela 1: Matriz de Cargas (Pattern Matrix)
loadings_df = pd.DataFrame(fa.loadings_, index=determinants, columns=['Fator IAT', 'Fator ISP', 'Fator NSP'])
loadings_df = loadings_df[['Fator ISP', 'Fator IAT', 'Fator NSP']]
display(stylize(loadings_df, "Tabela 1: Matriz de Cargas Fatoriais (Pattern Matrix)"))

# Tabela 2: Matriz de Correlação entre Fatores (Phi Matrix)
phi_df = pd.DataFrame(fa.phi_, index=['Indice IAT', 'Indice ISP', 'Indice NSP'], columns=['Indice IAT', 'Indice ISP', 'Indice NSP'])
phi_df = phi_df.loc[['Indice ISP', 'Indice IAT', 'Indice NSP'], ['Indice ISP', 'Indice IAT', 'Indice NSP']]
display(stylize(phi_df, "Tabela 2: Matriz de Correlação entre Fatores"))

# Tabela 3: Consistência Interna e Adequação
def get_alpha(data):
    corr = data.corr()
    n = data.shape[1]
    avg_corr = corr.values[np.triu_indices_from(corr.values, k=1)].mean()
    return (n * avg_corr) / (1 + (n - 1) * avg_corr)

# Cálculos de Adequação
kmo_all, kmo_model = calculate_kmo(df[determinants])
chi_square_value, p_value_bartlett = calculate_bartlett_sphericity(df[determinants])

summary_df = pd.DataFrame({
    'Métrica': [
        'KMO Global',
        'Teste de Bartlett (p-valor)',
        'Alfa de Cronbach (ISP)',
        'Alfa de Cronbach (IAT)',
        'Alfa de Cronbach (NSP)'
    ],
    'Valor': [
        kmo_model,
        p_value_bartlett,
        get_alpha(df[items_isp]),
        get_alpha(df[items_iat]),
        get_alpha(df[items_nsp])
    ]
})
display(summary_df.style.set_caption("<div style='font-size:16px; font-weight:bold; color:black; padding:10px;'>Tabela 3: Adequação e Consistência</div>").format(subset=['Valor'], precision=4))

# 7. PROCESSAMENTO E DOWNLOAD DOS ARQUIVOS

# Cálculo dos Índices Finais (médias simples)
df['Indice_ISP'] = df[items_isp].mean(axis=1)
df['Indice_IAT'] = df[items_iat].mean(axis=1)
df['Indice_NSP'] = df[items_nsp].mean(axis=1)

# Arquivo A: Dataset Bruto (41 itens apenas)
df[all_q_names].to_csv('dataset_simulado_41_itens.csv', index=False)

# Arquivo B: Dataset com Índices (41 itens + colunas calculadas)
df.to_csv('dataset_com_indices_calculados.csv', index=False)

print("\nGerando downloads automáticos...")
for filename in ['dataset_simulado_41_itens.csv', 'dataset_com_indices_calculados.csv']:
    files.download(filename)
    print(f"-> {filename} pronto.")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,Fator ISP,Fator IAT,Fator NSP
Q1,0.795,-0.002,0.017
Q2,0.795,-0.033,-0.033
Q3,0.783,0.053,-0.041
Q4,0.797,0.019,0.058
Q6,0.784,-0.040,0.001
Q30,0.772,-0.008,0.017
Q31,0.794,0.010,-0.014
Q15,-0.068,0.805,0.019
Q16,0.021,0.816,0.028
Q17,0.070,0.817,-0.046


,Indice ISP,Indice IAT,Indice NSP
Indice ISP,1.000,-0.028,-0.068
Indice IAT,-0.028,1.000,-0.021
Indice NSP,-0.068,-0.021,1.000


,Métrica,Valor
0,KMO Global,0.9211
1,Teste de Bartlett (p-valor),0.0000
2,Alfa de Cronbach (ISP),0.8987
3,Alfa de Cronbach (IAT),0.9153
4,Alfa de Cronbach (NSP),0.9111



Gerando downloads automáticos...


...

<IPython.core.display.Javascript object>

-> dataset_simulado_41_itens.csv pronto.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-> dataset_com_indices_calculados.csv pronto.
